# General queries: no context-based restrictions
This implies that among "works" are also included entities such as charcaters, song contests, objects, web platforms etc.

Moreover, to include more works in general, the variable "creator" do not include only humans. 

In [2]:
from pandas import *
from queryWikidata import query_wikidata

In [3]:
endpoint = "https://query.wikidata.org/sparql"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:105.0) Gecko/20100101 Firefox/105.0"

## 1. Count all the derivative works present on Wikidata

To do that, we rely on the following properties:
- P737 influenced by
- P144 based on
- P941 inspired by
- P8371 references work
- P4969 derivative work
- P58 screenwriter
- P1877 after a work by
- P674 characters
- P170 creator
- P50 author



### 1.1 Derivative works without definition of work and derivative work 

In [87]:
q1_derivative_works = """
SELECT (COUNT(DISTINCT ?derivativeWork)AS ?derivativeWorksCount)
WHERE {
    {
      VALUES ?p {wdt:P737 wdt:P144 wdt:P941 wdt:P8371 wdt:P4969} 
      ?derivativeWork ?p ?work .
    }UNION{
      ?derivativeWork wdt:P1877 ?creator .
    }
}
"""
derivative_works_df1 = query_wikidata(endpoint, q1_derivative_works, user_agent, True)
derivative_works_df1

,derivativeWorksCount
0,86683


### 1.2 Derivative works with work's definition only

In [88]:
q2_derivative_works = """
SELECT (COUNT(DISTINCT ?derivativeWork)AS ?derivativeWorksCount)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                       
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }
    
    {
      VALUES ?p {wdt:P737 wdt:P144 wdt:P941 wdt:P8371 wdt:P4969} 
      ?derivativeWork ?p ?work .
    }UNION{
      ?derivativeWork wdt:P1877 ?creator .
    }
}
"""
derivative_works_df2 = query_wikidata(endpoint, q2_derivative_works, user_agent, True)
derivative_works_df2

,derivativeWorksCount
0,44386


### 1.3 Derivative works with derivative work's definition only

In [89]:
q3_derivative_works = """
SELECT (COUNT(DISTINCT ?derivativeWork)AS ?derivativeWorksCount)
WHERE {
    {
      ?derivativeWork wdt:P170 ?creator .
     }UNION{
      ?derivativeWork wdt:P50 ?creator .                       
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?derivativeWork .
    }
    
    {
      VALUES ?p {wdt:P737 wdt:P144 wdt:P941 wdt:P8371 wdt:P4969} 
      ?derivativeWork ?p ?work .
    }UNION{
      ?derivativeWork wdt:P1877 ?creator .
    }
}
"""
derivative_works_df3 = query_wikidata(endpoint, q3_derivative_works, user_agent, True)
derivative_works_df3

,derivativeWorksCount
0,23660


### 1.4 Derivative works with definition of both work and derivative work

In [90]:
q4_derivative_works = """
SELECT (COUNT(DISTINCT ?derivativeWork)AS ?derivativeWorksCount)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                       
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    {
      ?derivativeWork wdt:P170 ?creator2 .  #I set creator with another variable since I don't want to include only people as defined by the previous union block
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                       
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }
    
    {
      VALUES ?p {wdt:P737 wdt:P144 wdt:P941 wdt:P8371 wdt:P4969} 
      ?derivativeWork ?p ?work .
    }UNION{
      ?derivativeWork wdt:P1877 ?creator .
    }
}
"""
derivative_works_df4 = query_wikidata(endpoint, q4_derivative_works, user_agent, True)
derivative_works_df4

,derivativeWorksCount
0,18347


## 2. Analyse the use of single properties to connect a product with its derivative work

### 2.1 Influenced by (`P737`)

#### 2.1.1 No definition of work and derivative work

In [91]:
q1_influenced_by = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInfluencedBy)
WHERE {

    ?derivativeWork wdt:P737 ?work . 
                              
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
influenced_by_df1 = query_wikidata(endpoint, q1_influenced_by, user_agent, True)
influenced_by_df1

,work,derivativeWorkInfluencedBy
0,10523,10249


#### 2.1.2 Only definition of work

In [93]:
q2_influenced_by = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInfluencedBy)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    ?derivativeWork wdt:P737 ?work . 
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
influenced_by_df2 = query_wikidata(endpoint, q2_influenced_by, user_agent, True)
influenced_by_df2

,work,derivativeWorkInfluencedBy
0,438,739


#### 2.1.3 Only definition of derivative work

In [94]:
q3_influenced_by = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInfluencedBy)
WHERE {
    {
      ?derivativeWork wdt:P170 ?creator .
     }UNION{
      ?derivativeWork wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P737 ?work . 
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
influenced_by_df3 = query_wikidata(endpoint, q3_influenced_by, user_agent, True)
influenced_by_df3

,work,derivativeWorkInfluencedBy
0,526,325


#### 2.1.4 Both definitions

In [97]:
q4_influenced_by = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInfluencedBy)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }
    
    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P737 ?work . 
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
influenced_by_df4 = query_wikidata(endpoint, q4_influenced_by, user_agent, True)
influenced_by_df4

,work,derivativeWorkInfluencedBy
0,185,172


### 2.2 Based on (`P144`)

#### 2.2.1 No definition of work and derivative work

In [99]:
q1_based_on = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkBasedOn)
WHERE {

    ?derivativeWork wdt:P144 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

based_on_df1 = query_wikidata(endpoint, q1_based_on, user_agent, True)
based_on_df1

,work,derivativeWorkBasedOn
0,34834,52736


#### 2.2.2 Only definition of work

In [100]:
q2_based_on = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkBasedOn)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    ?derivativeWork wdt:P144 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

based_on_df2 = query_wikidata(endpoint, q2_based_on, user_agent, True)
based_on_df2

,work,derivativeWorkBasedOn
0,18822,32141


#### 2.2.3 Only definition of derivative work

In [102]:
q3_based_on = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkBasedOn)
WHERE {
    {
      ?derivativeWork wdt:P170 ?creator .
     }UNION{
      ?derivativeWork wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P144 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

based_on_df3 = query_wikidata(endpoint, q3_based_on, user_agent, True)
based_on_df3

,work,derivativeWorkBasedOn
0,9461,12684


#### 2.2.4 Both definitions

In [103]:
q4_based_on = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkBasedOn)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }
    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P144 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

based_on_df4 = query_wikidata(endpoint, q4_based_on, user_agent, True)
based_on_df4

,work,derivativeWorkBasedOn
0,7494,10025


### 2.3 Inspired By (`P941`)

#### 2.3.1 No definition of work and derivative work

In [104]:
q1_inspired_by = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInspiredBy)
WHERE {

    ?derivativeWork wdt:P941 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

inspired_by_df1 = query_wikidata(endpoint, q1_inspired_by, user_agent, True)
inspired_by_df1

,work,derivativeWorkInspiredBy
0,5448,5285


#### 2.3.2 Only definition of work

In [105]:
q2_inspired_by = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInspiredBy)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    ?derivativeWork wdt:P941 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

inspired_by_df2 = query_wikidata(endpoint, q2_inspired_by, user_agent, True)
inspired_by_df2

,work,derivativeWorkInspiredBy
0,1601,1766


#### 2.3.3 Only definition of derivative work

In [106]:
q3_inspired_by = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInspiredBy)
WHERE {
    {
      ?derivativeWork wdt:P170 ?creator .
     }UNION{
      ?derivativeWork wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P941 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

inspired_by_df3 = query_wikidata(endpoint, q3_inspired_by, user_agent, True)
inspired_by_df3

,work,derivativeWorkInspiredBy
0,2281,2145


#### 2.3.4 Both definitions

In [107]:
q4_inspired_by = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInspiredBy)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }
    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P941 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

inspired_by_df4 = query_wikidata(endpoint, q4_inspired_by, user_agent, True)
inspired_by_df4

,work,derivativeWorkInspiredBy
0,892,956


### 2.4 References Work (`P8371`)

#### 2.4.1 No definition of work and derivative work

In [108]:
q1_references_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkReferencesWork)
WHERE {

    ?derivativeWork wdt:P8371 ?work .
                      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

references_work_df1 = query_wikidata(endpoint, q1_references_work, user_agent, True)
references_work_df1

,work,derivativeWorkReferencesWork
0,1250,591


#### 2.4.2 Only definition of work

In [109]:
q2_references_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkReferencesWork)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    ?derivativeWork wdt:P8371 ?work .
                      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

references_work_df2 = query_wikidata(endpoint, q2_references_work, user_agent, True)
references_work_df2

,work,derivativeWorkReferencesWork
0,605,364


#### 2.4.3 Only definition of derivative work

In [110]:
q3_references_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkReferencesWork)
WHERE {
    {
      ?derivativeWork wdt:P170 ?creator .
     }UNION{
      ?derivativeWork wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P8371 ?work .
                      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

references_work_df3 = query_wikidata(endpoint, q3_references_work, user_agent, True)
references_work_df3

,work,derivativeWorkReferencesWork
0,901,248


#### 2.4.4 Both definitions

In [111]:
q4_references_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkReferencesWork)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }
    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P8371 ?work .
                      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

references_work_df4 = query_wikidata(endpoint, q4_references_work, user_agent, True)
references_work_df4

,work,derivativeWorkReferencesWork
0,494,184


### 2.5 Derivative Work (`P4969`)

#### 2.5.1 No definition of work and derivative work

In [113]:
q1_derivative_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    ?work wdt:P4969 ?derivativeWork .

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

derivative_work_df1 = query_wikidata(endpoint, q1_derivative_work, user_agent, True)
derivative_work_df1

,work,derivativeWork
0,9265,15959


#### 2.5.2 Only definition of work

In [114]:
q2_derivative_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    ?work wdt:P4969 ?derivativeWork .

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

derivative_work_df2 = query_wikidata(endpoint, q2_derivative_work, user_agent, True)
derivative_work_df2

,work,derivativeWork
0,9265,15959


#### 2.5.3 Only definition of derivative work

In [117]:
q4_derivative_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }
    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }

    ?work wdt:P4969 ?derivativeWork .

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

derivative_work_df4 = query_wikidata(endpoint, q4_derivative_work, user_agent, True)
derivative_work_df4

,work,derivativeWork
0,3850,5084


#### 2.5.4 Both definitions

In [ ]:
q3_derivative_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork)
WHERE {
    {
      ?derivativeWork wdt:P170 ?creator .
     }UNION{
      ?derivativeWork wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?derivativeWork .
    }

    ?work wdt:P4969 ?derivativeWork .

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

derivative_work_df3 = query_wikidata(endpoint, q3_derivative_work, user_agent, True)
derivative_work_df3

### 2.6 After a Work By (`P1877`)

#### 2.6.1 No definition of work and derivative work

In [119]:
q1_after_work_by = """
SELECT (COUNT(DISTINCT ?creator) AS ?creatorCount) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkAfterWorkBy)
WHERE {

    ?derivativeWork wdt:P1877 ?creator .
      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

after_work_by_df1 = query_wikidata(endpoint, q1_after_work_by, user_agent, True)
after_work_by_df1

,creatorCount,derivativeWorkAfterWorkBy
0,4830,12188


#### 2.6.2 Only definition of work

In [120]:
q2_after_work_by = """
SELECT (COUNT(DISTINCT ?creator) AS ?creatorCount) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkAfterWorkBy)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    ?derivativeWork wdt:P1877 ?creator .
      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

after_work_by_df2 = query_wikidata(endpoint, q2_after_work_by, user_agent, True)
after_work_by_df2

,creatorCount,derivativeWorkAfterWorkBy
0,3898,10900


#### 2.6.3 Only definition of derivative work

In [122]:
q3_after_work_by = """
SELECT (COUNT(DISTINCT ?creator) AS ?creatorCount) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkAfterWorkBy)
WHERE {
    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P1877 ?creator .
      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

after_work_by_df3 = query_wikidata(endpoint, q3_after_work_by, user_agent, True)
after_work_by_df3

,creatorCount,derivativeWorkAfterWorkBy
0,1685,5103


#### 2.6.4 Both definitions

In [123]:
q4_after_work_by = """
SELECT (COUNT(DISTINCT ?creator) AS ?creatorCount) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkAfterWorkBy)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }
    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P1877 ?creator .
      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

after_work_by_df4 = query_wikidata(endpoint, q4_after_work_by, user_agent, True)
after_work_by_df4

,creatorCount,derivativeWorkAfterWorkBy
0,1401,4531
